In [130]:
import importlib
import pandas as pd
import numpy as np
import os
import py_vollib_vectorized
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import scipy

In [131]:
import classes, utils, iv_models
_ = importlib.reload(classes)
_ = importlib.reload(utils)
_ = importlib.reload(iv_models)


from classes import Ticker
from utils import create_market_state
from iv_models import BasicMidIVPolynomial, MidIV

In [132]:
t = Ticker("AXISBANK", expiry_date="20210624")
oc1 = t.get_option_chain('20210623 09:30:00')
oc2 = t.get_option_chain('20210623 11:30:00')
oc1.fit_iv_model(BasicMidIVPolynomial(2))
oc2.fit_iv_model(BasicMidIVPolynomial(2))

loading data from ./data/20210624/influxOP_AXISBANK21JUN_20210624.csv
loading data from ./data/20210623/influxOP_AXISBANK21JUN_20210623.csv


In [133]:
df, greeks, price = oc1.personal_prices_greeks([74000, 75000, 76000])
df

,strike,futPrice,moneyness,instrument,time_to_expiry,fit_iv
0,74000,74187.5,-0.002531,p,0.003938,0.330073
1,75000,74187.5,0.010892,c,0.003938,0.327872
2,76000,74187.5,0.024138,c,0.003938,0.336023


In [134]:
greeks

,delta,gamma1,gamma2,theta,rho,vega1,vega2
0,-0.447281,0.000360,0.000353,-107843.540690,-132.722861,2573.095109,2528.507704
1,0.301834,0.000198,0.000195,-58499.638072,87.032840,1405.142821,1386.954328
2,0.128357,0.000082,0.000081,-25495.765482,37.105304,597.545371,591.309692


In [135]:
price

,price
0,523.044371
1,289.851607
2,99.371704


In [129]:
# s/k vector
s_vec = df.futPrice.values
k_vec = df.strike.values
d_sk = np.log(np.divide(s_vec,k_vec))
nlen = df.futPrice.values.shape[0]
# interest rate
r = np.zeros(nlen)
# dividends
q = np.zeros(nlen)
sigma = df.fit_iv.values
t1 = df.time_to_expiry.values
# Calculate d1
d1 = (d_sk+((r-q+np.power(sigma,2)/2)*(t1)))/(sigma*np.power(t1,0.5))
# Calculate cdf of d1
cdf_d1 = scipy.stats.norm.cdf(d1)
df['cdf_d1'] = d1
# Delta in Put is CDF(-d1)
df['cdf_d1_signed']     = np.where(df['instrument'] == 'c', cdf_d1, (1-cdf_d1))
e_qt = np.exp(-1*q*t1)
e_rt = np.exp(-1*r*t1)
df['e_qt'] = e_qt
# Delta in Put has negative sign
df['e_qt_with_delta_sign']     = np.where(df['instrument'] == 'c', e_qt, -1*e_qt)
df['delta'] = df['e_qt_with_delta_sign']*df['cdf_d1_signed'] 


# Calculate d2 using d1
d2 = d1-sigma*np.power(t1,0.5)
cdf_d2 = scipy.stats.norm.cdf(d2)
df['cdf_d2'] =  cdf_d2
# use vega from wikipedia
vega1 = s_vec*e_qt*cdf_d1*np.power(t1,0.5)
vega2 = k_vec*e_rt*cdf_d2*np.power(t1,0.5)
df['vega1'] = vega1
df['vega2'] = vega2


# rho has negative d2 in put
df['cdf_d2_signed'] =  np.where(df['instrument'] == 'c', cdf_d2, (1-cdf_d2))
rho_unsigned = df.cdf_d2_signed.values * t1*k_vec*e_rt
df['rho_unsigned'] = rho_unsigned
# rho has a negative sign outside in put formula
df['rho'] =  np.where(df['instrument'] == 'c', rho_unsigned, -1*rho_unsigned)


# gamma - formula 1
df['gamma1'] = np.divide(e_qt*cdf_d1,s_vec*sigma*np.power(t1,0.5))
df['gamma2'] = np.divide(k_vec*e_rt*cdf_d2,sigma*np.power(t1,0.5)*np.power(s_vec,2))

theta_t1 = -1 * np.divide((e_qt*s_vec*cdf_d1*sigma),(2*np.power(t1,0.5)))

cdf_d2_signed = df['cdf_d2_signed'].values
cdf_d1_signed = df['cdf_d1_signed'].values
theta_t2 = -1*r*k_vec*e_rt*cdf_d2_signed
theta_t3 = q*s_vec*e_qt*cdf_d1_signed
theta_t23 = theta_t2+theta_t3

df['theta_t1'] = theta_t1
df['theta_t23'] = theta_t23
df['theta_t23_signed'] =  np.where(df['instrument'] == 'c', theta_t23, -1*theta_t23)

df['theta'] = df['theta_t23_signed'].values+theta_t1

price_unsigned = s_vec*np.exp((r-q)*t1)*cdf_d1_signed-k_vec*cdf_d2_signed
df['price_unsigned'] = price_unsigned
df['price_undiscounted'] =  np.where(df['instrument'] == 'c', price_unsigned, -1*price_unsigned)

df['price'] =  df.price_undiscounted.values*e_rt

df1 = df[['strike',	'futPrice',	'moneyness',	'instrument',	'time_to_expiry',	'fit_iv']]

df2 = df[['delta',	'gamma1','gamma2',	'theta',	'rho',	'vega1','vega2']]
df3 = df[['price']]
df3

,price
0,523.044371
1,289.851607
2,99.371704


In [117]:
price_unsigned = s_vec*np.exp((r-q)*t1)*cdf_d1_signed-k_vec*cdf_d2_signed

array([2528.50770392, 1386.95432803,  591.30969241])

In [93]:
d2

array([ 0.11182055, -0.53970617, -1.15527842])

In [ ]:
cdf_d1*np.power(t1,0.5)*e_qr * s_vec

In [29]:
greeks

,delta,gamma,theta,rho,vega
0,-0.447281,0.000257,-270.850556,-1.327229,18.409612
1,0.301834,0.000228,-213.765215,0.870329,16.228923
2,0.128357,0.000134,-93.327525,0.371053,9.758503


In [18]:
df, greeks, price = oc1.ivs_prices_greeks([74000, 75000, 76000])
df

,strike,futPrice,moneyness,instrument,time_to_expiry,fit_iv
0,74000,74187.5,-0.002531,p,0.003938,0.330073
1,75000,74187.5,0.010892,c,0.003938,0.327872
2,76000,74187.5,0.024138,c,0.003938,0.336023


In [19]:
greeks

,delta,gamma,theta,rho,vega
0,-0.447281,0.000257,-270.850556,-1.327229,18.409612
1,0.301834,0.000228,-213.765215,0.870329,16.228923
2,0.128357,0.000134,-93.327525,0.371053,9.758503


In [21]:
price

,Price
0,523.044371
1,289.851607
2,99.371704


In [ ]:
oc1.df[oc1.df['strike'] == 74000]

In [ ]:
oc2.df[oc2.df['strike'] == 74000]

In [ ]:
334.998764 + 18.258711 * (0.2152 - 0.227586) * 100 - 35.217181  -0.042049 * 365

In [ ]:
oc2.iv_parameters()

In [ ]:
oc1.iv_parameters()

In [ ]:
-0.042049 * 365

In [ ]:
delta_diff = res1['delta'] * (oc2.future_price() - oc1.future_price()) 
gamma_diff = res1['gamma'] * (res2['delta'] - res1['delta']) 
theta_diff = res1['theta'] * (oc1.time_to_expiry() - oc2.time_to_expiry())

print("delta_diff", delta_diff)
print("gamma_diff", gamma_diff)
print("theta_diff", theta_diff)

In [ ]:
s = 0

for i in range(len(oc1.iv_parameters())):
    s += res1['vega'] * res1[f'greek_{i}'] * (oc2.iv_parameters()[i] - oc1.iv_parameters()[i])
s

In [ ]:
8.010609 * (1.0 * (0.33292242 - 0.31478401) + 0.028021 * (-0.12406091 + 0.11291487) )